In [1]:
from google.colab import drive
drive.mount("content/")

Mounted at content/


In [2]:
# !ls content/MyDrive/Colab_Notebooks/image_dataset/oxford-iiit-pet/
# Directory of oxford-iiit-pet
dir = "content/MyDrive/Colab_Notebooks/image_dataset/oxford-iiit-pet/"

In [17]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.feature import hog
import os
import numpy as np
from skimage import io, color, feature, exposure
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from skimage.transform import resize
import joblib


In [104]:
# # Function to extract HOG features from an image and visualize it
def extract_hog_features(image):
    # Image resizing
    image = resize(image, (160,80))

    # Check if the image has an alpha channel (fourth channel)
    if image.shape[-1] == 4:
        # Convert RGBA to RGB by discarding the alpha channel
        image = image[:, :, :3]

    gray_image = color.rgb2gray(image)
    hog_features = feature.hog(
            gray_image, pixels_per_cell=(14,14),
            cells_per_block=(2, 2),
            orientations=9,
            block_norm='L2-Hys')  # Specify block_norm

    return hog_features

In [105]:
# Feature Extraction [(2 Dogs, 3 Cats)][400, 600]

# Dog - Cat
# List image folder and get annotation
images_folder =  f"{dir}images/images/"
file_labels = open(f"{dir}annotations/list.txt", 'r')
cat_folder = images_folder + "cats/"
dog_folder = images_folder + "dogs/"

# File Reading
data = file_labels.readline()

# Splitting Images into 2 Folders
# while data:
#   line = data.split(" ")
#   img_name = f"{line[0]}.jpg"
#   img_path = f"{images_folder}{img_name}"

#   try :
#     line[2] = int(line[2])
#     if line[2] == 1:
#       destination = cat_folder

#     elif line[2] == 2 :
#       destination = dog_folder

#     try:
#       shutil.move(img_path, destination)
#     except :
#       print(f"Cannot move {img_path}")

#   except :
#     pass

#   data = file_labels.readline()

# print("Finish Copying")


species = {
      0 : {},
      1 : {}

}

while data:
  line = data.split(" ")
  img_name = "_".join(line[0].split("_")[:-1])
  try :
    line[2] = int(line[2])
    species[line[2]-1][img_name] = species[line[2]-1].get(img_name, 0) + 1
  except :
    pass

  data = file_labels.readline()

print("Finish Extracting Labels")


# dog_list & cat_list
dog_list = []
cat_list = []

for kind, count in species[0].items():
    if len(cat_list) == 3 :
      break

    if count == 200 :
      cat_list.append(kind)

for kind, count in species[1].items():
    if len(dog_list) == 2 :
      break

    if count == 200 :
      dog_list.append(kind)


species[0] = cat_list
species[1] = dog_list

# Images List
image_list = []

# Feature Extraction
features_list = []
labels_list = []
binary_labels_list = []
label_species = 1

# Looping over Folders
for folder in os.scandir(images_folder):
    if folder.is_dir():
      if folder.name == 'cats':
        label = 0
      else :
        label = 1

      for name in species[label]:
        # Looping over Images
        for number in range(1, 250):
          # Improvement => make equally species images [Get equal numbers of each Breed so, it is not baised!]
          img_path = f"{folder.path}/{name}_{number}.jpg"
          print(f"Image : {img_path} is under pre-processing!")

          try :
            # Read Image
            image = io.imread(img_path)

            # Extract Features
            feature_list = extract_hog_features(image)

            # Append the label to the labels list
            features_list.append(feature_list)
            labels_list.append(label_species)
            binary_labels_list.append(label)
          except Exception as e :
            print(f"{str(e)} Error : {img_path}")

        label_species += 1


print("Finish Load")

Finish Extracting Labels
Image : content/MyDrive/Colab_Notebooks/image_dataset/oxford-iiit-pet/images/images/cats/Bengal_1.jpg is under pre-processing!
Image : content/MyDrive/Colab_Notebooks/image_dataset/oxford-iiit-pet/images/images/cats/Bengal_2.jpg is under pre-processing!
Image : content/MyDrive/Colab_Notebooks/image_dataset/oxford-iiit-pet/images/images/cats/Bengal_3.jpg is under pre-processing!
Image : content/MyDrive/Colab_Notebooks/image_dataset/oxford-iiit-pet/images/images/cats/Bengal_4.jpg is under pre-processing!
Image : content/MyDrive/Colab_Notebooks/image_dataset/oxford-iiit-pet/images/images/cats/Bengal_5.jpg is under pre-processing!
Image : content/MyDrive/Colab_Notebooks/image_dataset/oxford-iiit-pet/images/images/cats/Bengal_6.jpg is under pre-processing!
Image : content/MyDrive/Colab_Notebooks/image_dataset/oxford-iiit-pet/images/images/cats/Bengal_7.jpg is under pre-processing!
Image : content/MyDrive/Colab_Notebooks/image_dataset/oxford-iiit-pet/images/images/ca

In [106]:
# Convert lists to NumPy arrays
features_array = np.array(features_list)
labels_array = np.array(labels_list)
binary_labels_array = np.array(binary_labels_list)


print(features_array)
print(labels_array)

[[0.02009169 0.0169542  0.01576388 ... 0.16976824 0.13604667 0.03146918]
 [0.09124904 0.02729912 0.02379092 ... 0.10588629 0.08724799 0.08862948]
 [0.16007471 0.16325052 0.16412642 ... 0.03345051 0.09985814 0.06716205]
 ...
 [0.22632957 0.09789114 0.06471539 ... 0.10327615 0.15917013 0.25218279]
 [0.14781721 0.17349345 0.03494695 ... 0.19430296 0.19277437 0.06258281]
 [0.23806868 0.05527004 0.03898272 ... 0.2258406  0.2258406  0.18918654]]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 

In [107]:
# Reshaping Features!
features_np = features_array.reshape(len(features_array), -1)
print(features_np.shape)


(984, 1440)


In [108]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_np, labels_array, test_size=0.3,
                                                    stratify = labels_array,
                                                    random_state=42)
X_train, X_test, y_binary_train, y_binary_test = train_test_split(features_np, binary_labels_array, test_size=0.3,
                                                    stratify = labels_array,
                                                    random_state=42)

from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
model_liblinear = LogisticRegression(solver='liblinear', max_iter=1000)

# Train 5 classes using 2 models
model_liblinear.fit(X_train, y_train)
model.fit(X_train, y_train)


model_binary = LogisticRegression(max_iter=1000)
model_binary_liblinear = LogisticRegression(solver='liblinear', max_iter=1000)

# Train 2 classes using 2 models
model_binary.fit(X_train, y_binary_train)
model_binary_liblinear.fit(X_train, y_binary_train)

# Predict on 5 classes
y_pred = model.predict(X_test)
y_pred_liblinear = model_liblinear.predict(X_test)

# Predict on 2 Classes
y_binary_pred = model_binary.predict(X_test)
y_binary_pred_liblinear = model_binary_liblinear.predict(X_test)

In [109]:
from sklearn.metrics import confusion_matrix, accuracy_score

# Analysis on 5 classes prediction
cm = confusion_matrix(y_test, y_pred)
cm_liblinear = confusion_matrix(y_test, y_pred_liblinear)
print(f"Confusion Matrix Before Feature Scaling : \n{cm}")
print(" ")
print(f"Confusion Matrix liblinear Before Feature Scaling : \n{cm_liblinear}")

print("\n\n")

print(f"Accuracy Before Feature Scaling :  {accuracy_score(y_test, y_pred)}")
print(f"Accuracy liblinear Before Feature Scaling : {accuracy_score(y_test, y_pred_liblinear)}")

print("\n\n")

# Analysis on 2 classes prediction
cm_binary = confusion_matrix(y_binary_test, y_binary_pred)
cm_binary_liblinear = confusion_matrix(y_binary_test, y_binary_pred_liblinear)
print(f"Confusion Matrix Before Feature Scaling : \n{cm_binary}")
print(" ")
print(f"Confusion Matrix liblinear Before Feature Scaling : \n{cm_binary_liblinear}")

print("\n\n")

print(f"Accuracy Before Feature Scaling :  {accuracy_score(y_binary_test, y_binary_pred)}")
print(f"Accuracy liblinear Before Feature Scaling : {accuracy_score(y_binary_test, y_binary_pred_liblinear)}")

Confusion Matrix Before Feature Scaling : 
[[25  7 12  6 10]
 [10 19 13 12  6]
 [15 10 20  7  4]
 [ 6 16  2 25 11]
 [ 6  5  6 20 23]]
 
Confusion Matrix liblinear Before Feature Scaling : 
[[25  9 12  5  9]
 [11 17 11 14  7]
 [17 11 17  7  4]
 [ 9 14  2 25 10]
 [ 6  7  7 21 19]]



Accuracy Before Feature Scaling :  0.3783783783783784
Accuracy liblinear Before Feature Scaling : 0.34797297297297297



Confusion Matrix Before Feature Scaling : 
[[143  33]
 [ 42  78]]
 
Confusion Matrix liblinear Before Feature Scaling : 
[[143  33]
 [ 41  79]]



Accuracy Before Feature Scaling :  0.7466216216216216
Accuracy liblinear Before Feature Scaling : 0.75


In [110]:
# Scaling Features
sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Training Model of 5 classes
model.fit(X_train, y_train)
model_liblinear.fit(X_train, y_train)

# Predict on 5 classes
y_pred = model.predict(X_test)
y_pred_liblinear = model_liblinear.predict(X_test)

# Analysis on 5 classes prediction
cm = confusion_matrix(y_test, y_pred)
cm_liblinear = confusion_matrix(y_test, y_pred_liblinear)

print(f"Confusion Matrix After Feature Scaling : \n{cm}")
print(" ")
print(f"Confusion Matrix liblinear After Feature Scaling : \n{cm_liblinear}")

print("\n\n")

print(f"Accuracy After Feature Scaling :  {accuracy_score(y_test, y_pred)}")
print(f"Accuracy liblinear After Feature Scaling : {accuracy_score(y_test, y_pred_liblinear)}")


print("\n\n")
# Train 2 classes using 2 models
model_binary.fit(X_train, y_binary_train)
model_binary_liblinear.fit(X_train, y_binary_train)

# Predict on 2 Classes
y_binary_pred = model_binary.predict(X_test)
y_binary_pred_liblinear = model_binary_liblinear.predict(X_test)

# Analysis on 2 classes prediction
cm_binary = confusion_matrix(y_binary_test, y_binary_pred)
cm_binary_liblinear = confusion_matrix(y_binary_test, y_binary_pred_liblinear)
print(f"Confusion Matrix After Feature Scaling : \n{cm_binary}")
print(" ")
print(f"Confusion Matrix liblinear After Feature Scaling : \n{cm_binary_liblinear}")

print("\n\n")

print(f"Accuracy After Feature Scaling :  {accuracy_score(y_binary_test, y_binary_pred)}")
print(f"Accuracy liblinear After Feature Scaling : {accuracy_score(y_binary_test, y_binary_pred_liblinear)}")

Confusion Matrix After Feature Scaling : 
[[26  8 13  4  9]
 [10 18 16 12  4]
 [12 14 21  6  3]
 [ 6 15  2 22 15]
 [ 6  6  7 16 25]]
 
Confusion Matrix liblinear After Feature Scaling : 
[[24  6 16  5  9]
 [ 6 23 12 12  7]
 [13 12 19  5  7]
 [ 8 13  4 21 14]
 [ 5  9 11 18 17]]



Accuracy After Feature Scaling :  0.3783783783783784
Accuracy liblinear After Feature Scaling : 0.35135135135135137



Confusion Matrix After Feature Scaling : 
[[134  42]
 [ 45  75]]
 
Confusion Matrix liblinear After Feature Scaling : 
[[133  43]
 [ 44  76]]



Accuracy After Feature Scaling :  0.706081081081081
Accuracy liblinear After Feature Scaling : 0.706081081081081


In [16]:
# prompt: Kmeans classifier on previous features_array variable

from sklearn.cluster import KMeans
# Create a KMeans model with 3 clusters
kmeans = KMeans(n_clusters=5)

# Fit the model to the features
kmeans.fit(features_array)

# Predict the cluster labels for each data point
labels = kmeans.predict(features_array)

# Print the cluster labels
print(labels)


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


[1 2 0 2 2 2 2 2 0 2 2 2 3 1 2 1 3 0 3 0 3 3 2 0 3 3 0 0 2 3 1 4 2 2 2 3 3
 1 1 4 3 2 2 4 3 1 0 2 2 1 3 2 2 0 3 3 0 4 2 1 3 2 2 1 1 4 0 4 3 3 2 0 1 3
 1 4 4 2 1 1 0 1 2 3 2 0 0 2 3 2 4 3 1 2 4 2 2 4 2 1 2 1 0 2 2 0 1 2 0 1 2
 3 0 2 0 1 0 1 0 3 3 2 4 1 1 2 4 0 2 2 2 2 1 2 2 3 2 1 1 2 3 0 1 4 2 2 3 1
 1 2 2 1 4 3 3 3 1 3 4 1 3 0 3 1 1 1 2 4 2 0 0 2 2 1 1 2 3 2 0 2 1 2 1 1 2
 2 2 3 2 0 0 3 1 2 1 2 1 4 2 1 2 0 3 2 2 1 3 4 2 0 1 0 3 3 2 3 2 4 4 0 2 2
 3 2 1 3 3 2 0 1 3 4 1 2 0 2 2 2 0 2 3 3 4 3 3 4 0 1 1 0 0 1 3 2 0 2 3 2 1
 1 2 3 4 1 1 1 3 3 2 2 1 4 3 4 1 2 3 0 0 2 4 2 0 1 1 1 2 0 0 4 1 3 1 3 1 2
 3 0 2 4 3 2 3 4 1 1 4 3 0 1 1 1 1 1 0 2 3 1 1 4 4 4 2 3 1 0 0 4 2 4 0 3 3
 3 1 2 3 3 2 2 0 1 0 2 2 3 2 3 1 3 1 2 3 1 0 3 0 2 1 2 3 2 2 2 4 1 2 3 1 1
 0 4 0 1 4 0 3 3 2 1 1 1 1 1 0 1 3 1 2 2 4 1 2 2 1 1 2 1 0 1 1 3 2 0 1 3 1
 2 0 2 1 2 0 0 1 3 2 1 4 2 2 2 2 4 4 2 4 3 3 1 4 1 3 3 0 3 0 2 1 1 4 3 3 1
 1 3 1 4 0 4 4 3 0 1 0 0 0 0 4 3 3 0 4 2 1 1 0 2 3 3 1 1 4 2 4 1 4 3 0 0 0
 2 2 3 1 1 0 2 1 2 3 4 0 